In [1]:
# organize imports
from __future__ import print_function

# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

#imports
import numpy as np
import pickle
import sys

# keras imports
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras import backend as K

# S3 imports
from io import BytesIO
import boto3, os

Using TensorFlow backend.


## Loading file from S3

In [2]:
with open("../rootkey.csv") as f:
    ACCESS_ID = f.readline().strip().split('=')[1]
    ACCESS_KEY = f.readline().strip().split('=')[1]

In [3]:
s3 = boto3.resource('s3', 
                    aws_access_key_id=ACCESS_ID,
                    aws_secret_access_key= ACCESS_KEY)

myBucket = s3.Bucket('hackathon-nissan')

In [14]:
with BytesIO() as data:
    myBucket.download_fileobj("classifier.pickle", data)
    data.seek(0)    # move back to the beginning after writing
    classifier = pickle.load(data)

### Testing the loaded classifier

In [15]:
# get all the train labels
train_labels = ["scratch", "no-scratch"]
base_model = InceptionV3(include_top=False, weights="imagenet", input_tensor=Input(shape=(299,299,3)))
model = Model(input=base_model.input, output=base_model.get_layer('custom').output)
image_size = (299, 299)
img = image.load_img("/home/ec2-user/damage360/datasets/demo-photos/damaged/20181005_165421.jpg", target_size=image_size)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
feature = model.predict(x)
flat = feature.flatten()
flat = np.expand_dims(flat, axis=0)
preds = classifier.predict(flat)
prediction = train_labels[preds[0]]

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("cu...)`
  """


In [17]:
prediction

'scratch'

## Uploading file to S3

### From Python variable

### From file

In [4]:
import math, os
import boto
from filechunkio import FileChunkIO

# Connect to S3
s3_connection = boto.connect_s3(aws_access_key_id=ACCESS_ID, aws_secret_access_key=ACCESS_KEY)
bucket = s3_connection.get_bucket('hackathon-nissan')

In [9]:
# Get file info
source_path = 'model-prediction/features-models.h5'
source_size = os.stat(source_path).st_size

In [10]:
# Create a multipart upload request
mp = bucket.initiate_multipart_upload(os.path.basename(source_path))

In [11]:
# Use a chunk size of 50 MiB (feel free to change this)
chunk_size = 52428800
chunk_count = int(math.ceil(source_size / float(chunk_size)))

In [12]:
# Send the file parts, using FileChunkIO to create a file-like object
# that points to a certain byte range within the original file. We
# set bytes to never exceed the original file size.
for i in range(chunk_count):
    offset = chunk_size * i
    bytes = min(chunk_size, source_size - offset)
    with FileChunkIO(source_path, 'r', offset=offset,
                     bytes=bytes) as fp:
        mp.upload_part_from_file(fp, part_num=i + 1)

# Finish the upload
mp.complete_upload()

<CompleteMultiPartUpload: hackathon-nissan.features-models.h5>

In [33]:
# for bucket in s3.buckets.all():
#     print(bucket.name)

cdd-damage360
sfbayareabikeshare


In [34]:
# for obj in s3.Bucket(name='cdd-damage360').objects.all():
#     print(os.path.join(obj.bucket_name, obj.key))

cdd-damage360/classifier.pickle


In [36]:
# from urllib.request import urlopen
# myurl = "https://s3-us-west-2.amazonaws.com/cdd-damage360/classifier.pickle"
# classifier = urlopen(myurl)